In [3]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QPushButton, QLabel, QLineEdit, QFileDialog, QMessageBox
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas

class LogAnalyzerApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.initUI()
        self.data_loaded = False

    def initUI(self):
        self.setWindowTitle("Log Analyzer")
        self.setGeometry(100, 100, 800, 600)

        main_widget = QWidget(self)
        self.setCentralWidget(main_widget)

        layout = QVBoxLayout()
        main_widget.setLayout(layout)

        self.canvas = FigureCanvas(plt.figure())
        layout.addWidget(self.canvas)

        self.path_label = QLabel("Enter Log File Path:")
        layout.addWidget(self.path_label)

        self.path_input = QLineEdit()
        layout.addWidget(self.path_input)

        self.browse_button = QPushButton("Browse")
        self.browse_button.clicked.connect(self.browseFile)
        layout.addWidget(self.browse_button)

        self.load_button = QPushButton("Load File")
        self.load_button.clicked.connect(self.loadFile)
        layout.addWidget(self.load_button)

        self.status_label = QLabel()
        layout.addWidget(self.status_label)

    def browseFile(self):
        file_dialog = QFileDialog(self)
        file_path, _ = file_dialog.getOpenFileName(self, "Open Log File", "", "Text Files (*.txt);;All Files (*)")
        if file_path:
            self.path_input.setText(file_path)

    def loadFile(self):
        file_path = self.path_input.text()
        if file_path:
            try:
                self.data = pd.read_csv(file_path, delimiter='\t')  # Change delimiter if necessary
                self.data_loaded = True
                self.plotData()
            except Exception as e:
                self.status_label.setText("Error loading file: " + str(e))
        else:
            self.status_label.setText("Please enter a file path.")

    def plotData(self):
        if self.data_loaded:
            self.canvas.figure.clear()
            ax = self.canvas.figure.add_subplot(111)
            
            # Plot data here
            # Example: ax.plot(self.data['time'], self.data['value'])

            # Calculate trend line and threshold
            # Example: trend_line = np.polyfit(self.data['time'], self.data['value'], 1)
            #          threshold_time = ...  # Calculate threshold time

            # Plot trend line and threshold
            # Example: ax.plot(self.data['time'], np.polyval(trend_line, self.data['time']), label='Trend Line')
            #          ax.axhline(y=threshold_time, color='r', linestyle='--', label='Threshold')

            ax.legend()
            self.canvas.draw()

            # Display message based on threshold time
            if threshold_time < 30:
                message = "Preventive maintenance required."
            else:
                message = "PM could be done in next round."

            self.status_label.setText(message)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = LogAnalyzerApp()
    window.show()
    sys.exit(app.exec_())



SystemExit: 0

<Figure size 782x454 with 0 Axes>

In [2]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QPushButton, QLabel, QLineEdit, QFileDialog, QMessageBox
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from datetime import datetime, timedelta

class LogAnalyzerApp(QMainWindow):
    def __init__(self):
        super().__init__()

        self.initUI()

    def initUI(self):
        self.setWindowTitle("Log Analyzer")
        self.setGeometry(100, 100, 800, 600)

        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)

        self.layout = QVBoxLayout()

        self.path_label = QLabel("Enter Log File Path:")
        self.path_edit = QLineEdit()
        self.browse_button = QPushButton("Browse")
        self.browse_button.clicked.connect(self.browseFile)

        self.layout.addWidget(self.path_label)
        self.layout.addWidget(self.path_edit)
        self.layout.addWidget(self.browse_button)

        self.canvas = FigureCanvas(Figure(figsize=(5, 3)))
        self.layout.addWidget(self.canvas)

        self.open_button = QPushButton("Open File")
        self.open_button.clicked.connect(self.openFile)
        self.layout.addWidget(self.open_button)

        self.analyze_button = QPushButton("Analyze")
        self.analyze_button.clicked.connect(self.analyzeData)
        self.layout.addWidget(self.analyze_button)

        self.result_label = QLabel("")
        self.layout.addWidget(self.result_label)

        self.central_widget.setLayout(self.layout)

    def browseFile(self):
        file_dialog = QFileDialog(self)
        file_path, _ = file_dialog.getOpenFileName(self, "Open Log File", "", "Text Files (*.txt);;All Files (*)")
        self.path_edit.setText(file_path)

    def openFile(self):
        file_path = self.path_edit.text()
        try:
            self.data = pd.read_csv(file_path, sep='\t')  # Assuming tab-separated data
            self.result_label.setText("File opened successfully.")
        except Exception as e:
            self.result_label.setText(f"Error opening file: {str(e)}")

    def analyzeData(self):
        try:
            # Perform data processing here
            # For example, assuming 'timestamp' is a column in your DataFrame
            self.data['timestamp'] = pd.to_datetime(self.data['timestamp'])
            self.data.sort_values(by='timestamp', inplace=True)
            
            # Plot data
            ax = self.canvas.figure.add_subplot(111)
            ax.plot(self.data['timestamp'], self.data['value'], marker='o', linestyle='-')
            ax.set_xlabel('Time')
            ax.set_ylabel('Value')
            ax.set_title('Data Plot')
            
            # Fit a trend line
            trend = np.polyfit(np.arange(len(self.data)), self.data['value'], 1)
            trend_fn = np.poly1d(trend)
            ax.plot(self.data['timestamp'], trend_fn(np.arange(len(self.data))), linestyle='--', color='red')
            
            # Calculate time to reach threshold
            threshold = 100  # Set your threshold value here
            days_to_threshold = np.interp(threshold, trend_fn(np.arange(len(self.data))), np.arange(len(self.data)))
            days_to_threshold = max(0, days_to_threshold)  # Ensure positive value
            
            if days_to_threshold < 30:
                message = f"Preventive maintenance required in {days_to_threshold:.2f} days."
            else:
                message = f"PM could be done in the next round."
            
            self.result_label.setText(message)
            self.canvas.draw()
            
        except Exception as e:
            self.result_label.setText(f"Error analyzing data: {str(e)}")

def main():
    app = QApplication(sys.argv)
    window = LogAnalyzerApp()
    window.show()
    sys.exit(app.exec_())

if __name__ == "__main__":
    main()


SystemExit: 0